In [0]:
import pyspark
from pyspark import SparkContext, SparkConf
siva = SparkConf().setAppName("Mini Project Flight_data")
sc = SparkContext.getOrCreate(conf=siva)
from pyspark.sql import SparkSession

In [0]:
flights = spark.read.options(inferSchema='True', header='True', delimiter=',').csv('s3://flightdata-cdb/2007.csv')

In [0]:
flights.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- C

1.Find the mot frequent tail number which is getting in destination by maximum

In [0]:
dp=flights

In [0]:
freq_tailnum=dp.na.drop(subset=["TailNum"])

In [0]:
# /For the counter method for like() in spark dataframe I have used the negation operator (~) instead of the 'not' keyword in the filter.
freq_tailnum.groupBy("Dest","TailNum").count().orderBy("count",ascending=False).filter(~dp["TailNum"].like("0%")).show()

+----+-------+-----+
|Dest|TailNum|count|
+----+-------+-----+
| HNL| N655BR| 2241|
| HNL| N651BR| 2173|
| HNL| N654BR| 2138|
| HNL| N693BR| 2067|
| HNL| N479HA| 2038|
| HNL| N478HA| 2024|
| HNL| N485HA| 1984|
| HNL| N480HA| 1976|
| HNL| N484HA| 1944|
| HNL| N487HA| 1909|
| HNL| N481HA| 1868|
| HNL| N810AL| 1843|
| HNL| N477HA| 1837|
| HNL| N837AL| 1836|
| HNL| N475HA| 1816|
| HNL| N486HA| 1787|
| HNL| N646BR| 1768|
| HNL| N836AL| 1761|
| HNL| N824AL| 1754|
| HNL| N808AL| 1740|
+----+-------+-----+
only showing top 20 rows



2.Find out the cancelled flight  details for the last quarter of the year 2007

In [0]:
req_month = [10,11,12]
cancelled_flight = dp.select("Month","TailNum","FlightNum").filter((dp.Cancelled == "1") & (dp.Month.isin(req_month))).orderBy("Month").show()

+-----+-------+---------+
|Month|TailNum|FlightNum|
+-----+-------+---------+
|   10|      0|      195|
|   10|      0|      710|
|   10|      0|     1191|
|   10|      0|      514|
|   10|      0|     1799|
|   10|      0|       30|
|   10|      0|      507|
|   10|      0|      449|
|   10|      0|      820|
|   10|      0|     1036|
|   10|      0|     1243|
|   10|      0|     1804|
|   10|      0|     2010|
|   10|      0|     1235|
|   10|      0|     1970|
|   10|      0|      336|
|   10|      0|      507|
|   10|      0|     2843|
|   10|      0|      162|
|   10|      0|     2980|
+-----+-------+---------+
only showing top 20 rows



3.Find out the average weather delays for a particular flight per month

In [0]:
avg_weather = dp.select("Month","WeatherDelay","FlightNum").groupBy("Month","FlightNum").mean(("WeatherDelay")).orderBy("Month","FlightNum")

In [0]:
avg_weather.withColumnRenamed("avg(WeatherDelay)","Average_WeatherDelay").show()

+-----+---------+--------------------+
|Month|FlightNum|Average_WeatherDelay|
+-----+---------+--------------------+
|    1|        1| 0.40476190476190477|
|    1|        2| 0.05555555555555555|
|    1|        3|                 0.0|
|    1|        4|                 0.0|
|    1|        5|  0.8608695652173913|
|    1|        6|  0.7372262773722628|
|    1|        7|  0.7966101694915254|
|    1|        8|  0.5598591549295775|
|    1|        9|  0.7589285714285714|
|    1|       10| 0.10507246376811594|
|    1|       11|                 0.0|
|    1|       12| 0.07894736842105263|
|    1|       14|                 0.0|
|    1|       15| 0.04365079365079365|
|    1|       16|  0.3196480938416422|
|    1|       17| 0.15819209039548024|
|    1|       18|                 0.0|
|    1|       19| 0.33807829181494664|
|    1|       20|  0.2111111111111111|
|    1|       21|  0.8206521739130435|
+-----+---------+--------------------+
only showing top 20 rows



4.Inspite of NASDelay, SecurityDelay, LateAircraftDelay,Weatherdealy which flight reached on time

In [0]:
On_time=dp.select("FlightNum").filter(((dp.CarrierDelay>0) | (dp.WeatherDelay>0) | (dp.NASDelay>0) | (dp.SecurityDelay>0) | (dp.LateAircraftDelay>0)) & (dp.ArrDelay <= 0)).show()

+---------+
|FlightNum|
+---------+
+---------+



5.Month wise total distance travelled by each flight number in every month

In [0]:
from pyspark.sql.functions import sum

In [0]:
Monthwise=dp.groupBy("FlightNum","Month").agg({"Distance": "sum"}).orderBy("Month").dropDuplicates(["FlightNum"]).withColumnRenamed("sum(Distance)","Sum_Distance").show(280)

+---------+-----+------------+
|FlightNum|Month|Sum_Distance|
+---------+-----+------------+
|     6654|    1|       48832|
|      471|    1|      273506|
|     6620|    1|       29736|
|     2866|    1|       34992|
|     7240|    1|         157|
|     2122|    1|       42411|
|     1959|    1|       47698|
|      148|    1|      264623|
|     5300|    1|       23746|
|     2659|    1|      110178|
|     4519|    1|       32388|
|     5156|    1|       31877|
|     1238|    1|      126366|
|     6336|    1|        5050|
|     3794|    1|       50257|
|     7253|    1|        6792|
|     4101|    1|       24056|
|      496|    1|      222590|
|     1829|    1|      116843|
|     5518|    1|       13981|
|     4935|    1|       27060|
|     6658|    1|       17453|
|     4900|    1|       47360|
|      833|    1|      151265|
|     5803|    1|        4373|
|     2142|    1|       25936|
|     1645|    1|      207737|
|     3918|    1|       35681|
|     4818|    1|       28856|
|      4

6.Month wise how many flights get diverted(origin to destination)

In [0]:
diverted = dp.filter(dp.Diverted == "1").groupBy("Month").count().orderBy("Month").withColumnRenamed("count","Count_Diverted").show()

+-----+--------------+
|Month|Count_Diverted|
+-----+--------------+
|    1|          1200|
|    2|          1261|
|    3|          1275|
|    4|          1193|
|    5|          1442|
|    6|          2199|
|    7|          2150|
|    8|          2101|
|    9|           962|
|   10|          1000|
|   11|           881|
|   12|          1515|
+-----+--------------+



7.Week and month wise number of trips in all the flights

In [0]:
from pyspark.sql.functions import count
from pyspark.sql.functions import when

In [0]:
noofweek = dp.withColumn("Week",when((dp.DayofMonth >= 1) & (dp.DayofMonth <= 7), 1).when((dp.DayofMonth >= 8) & (dp.DayofMonth <= 14), 2).when((dp.DayofMonth >= 15) & (dp.DayofMonth <= 21), 3).when((dp.DayofMonth >= 22) & (dp.DayofMonth <= 28), 4).when((dp.DayofMonth >= 29) & (dp.DayofMonth <= 31), 5).otherwise('end'))


In [0]:
noofweek.filter(dp.Cancelled == 0).groupBy('Month','Week','FlightNum').agg(count('FlightNum').alias("No_Of_Flights")).orderBy('Month', 'Week').show(250)

+-----+----+---------+-------------+
|Month|Week|FlightNum|No_Of_Flights|
+-----+----+---------+-------------+
|    1|   1|      827|           25|
|    1|   1|     4145|           14|
|    1|   1|     4468|            7|
|    1|   1|      710|           63|
|    1|   1|     4712|           18|
|    1|   1|     1855|           38|
|    1|   1|     1392|            7|
|    1|   1|     1293|           46|
|    1|   1|     1420|           48|
|    1|   1|      131|           51|
|    1|   1|      436|           71|
|    1|   1|      207|           60|
|    1|   1|     2034|           39|
|    1|   1|     7273|           12|
|    1|   1|     2789|           15|
|    1|   1|     5116|           13|
|    1|   1|     2662|           24|
|    1|   1|     5035|            6|
|    1|   1|     2236|           12|
|    1|   1|     5162|            5|
|    1|   1|      965|           18|
|    1|   1|     6599|            7|
|    1|   1|      343|           73|
|    1|   1|     6831|            7|
|

8.Which flights covered maximum origin and destination by month wise

In [0]:
from pyspark.sql.functions import countDistinct, desc
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

In [0]:
MaxOrigin = dp.filter(dp.Cancelled == 0).groupBy('Month','TailNum').agg(count('Origin').alias("Max_Origins")).sort('Month', desc('Max_Origins'))

In [0]:
windowDept = Window.partitionBy('Month').orderBy(col("Max_Origins").desc())

In [0]:
MaxOrigin.withColumn("row",row_number().over(windowDept)).filter(col("row") == 1).drop("row").show()

+-----+-------+-----------+
|Month|TailNum|Max_Origins|
+-----+-------+-----------+
|    1| N828AL|        415|
|    2| N478HA|        382|
|    3| N808AL|        421|
|    4| N837AL|        421|
|    5| N836AL|        425|
|    6| N655BR|        416|
|    7| N655BR|        416|
|    8| N479HA|        404|
|    9| N654BR|        411|
|   10| N655BR|        451|
|   11| N475HA|        399|
|   12| N485HA|        401|
+-----+-------+-----------+



9.Average month wise arrival delay (flightnum wise)

In [0]:
from pyspark.sql.types import IntegerType

In [0]:
avg_arrdelay =dp.withColumn("ArrDelay", dp["ArrDelay"].cast(IntegerType())).groupby("Month","FlightNum").mean("ArrDelay").withColumnRenamed("avg(ArrDelay)","Average_ArrDelay").orderBy("Month").show()

+-----+---------+-------------------+
|Month|FlightNum|   Average_ArrDelay|
+-----+---------+-------------------+
|    1|     1208|  6.616788321167883|
|    1|     2545| -4.477611940298507|
|    1|      587|  4.144927536231884|
|    1|     1702| 6.8343558282208585|
|    1|     2483|  8.485714285714286|
|    1|     1720| 10.755102040816327|
|    1|     1677|  11.89655172413793|
|    1|      869|  7.234939759036145|
|    1|     1892|-3.2244897959183674|
|    1|     2324|  5.978260869565218|
|    1|     3452|               12.0|
|    1|     2667|  11.26086956521739|
|    1|     2202|  3.303030303030303|
|    1|     2086| 11.380952380952381|
|    1|     2870|            6.65625|
|    1|     7340| 16.479166666666668|
|    1|     5501| 23.916666666666668|
|    1|     4979|              41.25|
|    1|     5668|  5.903225806451613|
|    1|     4949|               10.0|
+-----+---------+-------------------+
only showing top 20 rows



10.Average month wise departure delay (flightnum wise)

In [0]:
avg_Depdelay =dp.withColumn("DepDelay", dp["DepDelay"].cast(IntegerType())).groupby("Month","FlightNum").mean("DepDelay").withColumnRenamed("avg(DepDelay)","Average_DepDelay").orderBy("Month").show()

+-----+---------+------------------+
|Month|FlightNum|  Average_DepDelay|
+-----+---------+------------------+
|    1|     1208| 8.094890510948906|
|    1|     2545| 5.462686567164179|
|    1|      587| 6.336956521739131|
|    1|     1702| 7.805471124620061|
|    1|     2483| 6.814285714285714|
|    1|     1720| 11.01530612244898|
|    1|     1677|14.131428571428572|
|    1|      869| 6.867469879518072|
|    1|     1892| 4.377551020408164|
|    1|     2324| 5.641304347826087|
|    1|     3452|               3.0|
|    1|     2667|12.347826086956522|
|    1|     2202| 8.090909090909092|
|    1|     2086|             11.25|
|    1|     2870|           3.71875|
|    1|     7340|           15.8125|
|    1|     5501|10.416666666666666|
|    1|     4979|              31.0|
|    1|     5668| 7.225806451612903|
|    1|     4949| 4.461538461538462|
+-----+---------+------------------+
only showing top 20 rows

